# Diffusion-Shock Inpainting
Diffusion-shock inpainting (DS) is a technique to fill in missing structures in images, developed in ["Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://link.springer.com/chapter/10.1007/978-3-031-31975-4_45) and the follow-up paper ["Regularised Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://arxiv.org/abs/2309.08761). In this notebook, we will first look at DS in the same setting as considered in these two papers, namely applied to images on $\mathbb{R}^2$. We will subsequently attempt to extend it to orientation scores in  $SE(2)$ (maybe even $SE(3)$ eventually?).

We can describe DS in a PDE-based formulation as
$$
\partial_t u = g(\lvert \nabla (G_{\nu} * u) \rvert^2) \underbrace{\Delta u}_{\textrm{Diffusion}} - \left(1 - g(\lvert \nabla (G_{\nu} * u) \rvert^2)\right) \underbrace{\mathrm{sgn}(\partial_{\vec{w} \vec{w}} (G_{\sigma} * u)) \lvert \nabla u \rvert}_{\textrm{Shock}},
$$
in which $g: [0, \infty) \to (0, 1]$ is a decreasing function with $g(0) = 1$, $G_{\alpha}$ is a Gaussian with standard deviation $\alpha$, and $\vec{w}$ is the dominant eigenvector of the structure tensor. It is clear then that $g$ switches between applying diffusion and shock: if the gradient of the image is small, we mostly apply diffusion, but if the gradient is large, we mostly apply shock. This makes sense, since a large gradient implies that there is a feature there, which we would like to sharpen up. 

The signum in the shock term switches between erosion and dilation. If the second derivative with respect to the dominant eigenvector of the structure tensor is positive, then we perform erosion (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$); otherwise we perform dilation (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$). In regularised DS, the signum is replaced with a soft signum, so that the selection of erosion vs dilation is less sensitive to noise. The signum of the second derivative of the dominant eigenvector of the structure tensor is not unlike the convexity criterion we know from studying vesselness; perhaps we could replace it?

What is the correct way to extend DS to $SE(2)$? It would make sense to keep the gradients and Laplacian. For the selection of erosion vs dilation we could again look at the vesselness convexity criterion. For switching between diffusion and shock, we could maybe use some sort of line/edge detector.

In [ ]:
import taichi as ti
ti.init(arch=ti.gpu, debug=False)
import numpy as np
# from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
# %matplotlib widget

## $\mathbb{R}^2$
We will start by reproducing the algorithm described in ["Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://link.springer.com/chapter/10.1007/978-3-031-31975-4_45) on $\mathbb{R}^2$.

In [ ]:
dim_I, dim_J = 256, 256
u = np.ones((dim_I, dim_J))
u[:, :25] = 0.
u[:, 100:160] = 0.
u[:, -25:] = 0.

u[100:160, :] = 0.5

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
orig = ax.imshow(u, origin="lower")
ax.set_title("$u$")
fig.colorbar(orig, ax=ax)
ax.set_xlim(0, u.shape[0]-1)
ax.set_ylim(0, u.shape[1]-1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$");

In [ ]:
import dsfilter

In [ ]:
u_filtered = dsfilter.DS_filter_R2(u, 3, 3, 2, 1., 1000)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
orig = ax.imshow(u_filtered, origin="lower")
ax.set_title("$u$")
fig.colorbar(orig, ax=ax)
ax.set_xlim(0, u.shape[0]-1)
ax.set_ylim(0, u.shape[1]-1)
ax.set_xlabel("$x$")
ax.set_ylabel("$y$");